<a href="https://colab.research.google.com/github/ningxia202109/llm-learn/blob/main/python-langchain-tutorials/Ollama_4_agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Build an Agent**

https://python.langchain.com/v0.2/docs/tutorials/agents/

In [1]:
# Ollama on T4 GPU

!curl https://ollama.ai/install.sh | sh

!echo 'debconf debconf/frontend select Noninteractive' | sudo debconf-set-selections
!sudo apt-get update && sudo apt-get install -y cuda-drivers

import os
# Set LD_LIBRARY_PATH so the system NVIDIA library
os.environ.update({'LD_LIBRARY_PATH': '/usr/lib64-nvidia'})
os.environ.update({'OLLAMA_HOST': '0.0.0.0'})

!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb

import subprocess
import threading
import time
import socket

def iframe_thread(port):
    while True:
        time.sleep(0.5)
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        result = sock.connect_ex(('127.0.0.1', port))
        if result == 0:
            break
        sock.close()

    p = subprocess.Popen(["cloudflared", "tunnel", "--url", f"http://127.0.0.1:{port}"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    for line in p.stderr:
        l = line.decode()
        if "trycloudflare.com " in l:
            print("\n\n\n\n\n")
            print("running ollama server\n\n", l[l.find("http"):], end='')
            print("\n\n\n\n\n")

threading.Thread(target=iframe_thread, daemon=True, args=(11434,)).start()

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0>>> Downloading ollama...
100 10975    0 10975    0     0  23646      0 --:--:-- --:--:-- --:--:-- 23653
############################################################################################# 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InReleas

In [13]:
!ollama serve > ollama-server.log 2>&1 &
!ollama run gemma2 > gemma2.log 2>&1 &
!sleep 120
!ollama list

NAME         	ID          	SIZE  	MODIFIED       
gemma2:latest	ff02c3702f32	5.4 GB	32 seconds ago	
qwen2:latest 	e0d4e1163c58	4.4 GB	6 minutes ago 	


In [3]:
# Install packages

!pip install -U -q langchain-community langgraph langchain-anthropic tavily-python langchain_openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.0/89.0 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.6/983.6 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 355.8/355.8 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 863.9/863.9 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.3/328.3 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

In [4]:
import os
from google.colab import userdata

os.environ["TAVILY_API_KEY"] = userdata.get('TAVILY_API_KEY')

In [5]:
# Define tools

from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in SF")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

[{'url': 'http://www.sfchronicle.com/weather-forecast/article/thunderstorms-lightning-tropical-california-19530762.php', 'content': 'Temperatures will be seasonably cool, in the upper 50s in the Sunset and Richmond districts, the low 60s in the Marina and Panhandle, and mid-60s in SoMa, downtown and the Mission. Most Popular Willie Mays’ childhood home in Alabama in poor repair, but could be restored Yosemite trail to Half Dome will be open limited hours starting in July McDonald’s at S.F.’s Stonestown mall closes after 30 years, blaming rent and higher wages Major Bay Area tech company to lay off 23% of global workforce Did progressives just lose the battle for Oakland’s political soul?  Temperatures in the Bay Area will be around seasonal averages this week, with highs near 60 at the coast and in the 70s to low 80s near the bay shoreline and 80s to low 90s inland. Weather models predict a few weak thunderstorms across the Bay Area and Central Valley on Tuesday afternoon, with stronge

In [14]:
# Use ChatOllama as a call tool
# https://python.langchain.com/v0.2/docs/integrations/chat/ollama/

from langchain_community.chat_models import ChatOllama
from langchain_core.messages import HumanMessage

llm = ChatOllama(model="gemma2")

response = llm.invoke([HumanMessage(content="hi!")])
response

AIMessage(content='Hi there! 👋 \n\nWhat can I do for you today? 😊', response_metadata={'model': 'gemma2', 'created_at': '2024-07-09T03:08:54.747075241Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 877701463, 'load_duration': 40365271, 'prompt_eval_count': 11, 'prompt_eval_duration': 88729000, 'eval_count': 16, 'eval_duration': 625148000}, id='run-733454e4-0fe2-4788-8182-1c69b00fb2a9-0')

In [15]:
# Ollama chat model does not support tool calling https://python.langchain.com/v0.1/docs/integrations/chat/
# But ChatOpenAI support Ollama, replace ChatOllama by ChatOpenAI
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    api_key="ollama",
    model="gemma2",
    base_url="http://localhost:11434/v1",
)

model_with_tools = model.bind_tools(tools)
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Hi there! 👋

How can I help you today? 😊
ToolCalls: []


In [16]:
response = model_with_tools.invoke([HumanMessage(content="What's the weather in SF?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: I don't have access to real-time information, including weather updates. To get the current weather in San Francisco, I recommend checking a reliable weather website or app.
ToolCalls: []


In [17]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

In [18]:
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]

[HumanMessage(content='hi!', id='6e7ae2b8-37e2-427b-a267-829de2035b34'),
 AIMessage(content='Hello! 👋 How can I help you today? 😄', response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 11, 'total_tokens': 23}, 'model_name': 'gemma2', 'system_fingerprint': 'fp_ollama', 'finish_reason': 'stop', 'logprobs': None}, id='run-4c363730-7e1c-4b42-9a48-4c74c3a27a6a-0', usage_metadata={'input_tokens': 11, 'output_tokens': 12, 'total_tokens': 23})]

In [19]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
)
response["messages"]

[HumanMessage(content='whats the weather in sf?', id='d266e63c-7051-4346-acf6-a5be13cde6e6'),
 AIMessage(content='I do not have access to real-time information, including weather updates. To get the current weather in San Francisco, I recommend checking a reliable weather app or website.', response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 15, 'total_tokens': 50}, 'model_name': 'gemma2', 'system_fingerprint': 'fp_ollama', 'finish_reason': 'stop', 'logprobs': None}, id='run-46c3db06-8700-4ccd-b815-f3850858e85f-0', usage_metadata={'input_tokens': 15, 'output_tokens': 35, 'total_tokens': 50})]

In [20]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='I do not have access to real-time information, including weather updates. To get the current weather in San Francisco, I recommend checking a reliable weather app or website.', response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 15, 'total_tokens': 50}, 'model_name': 'gemma2', 'system_fingerprint': 'fp_ollama', 'finish_reason': 'stop', 'logprobs': None}, id='run-7dbc979d-bcb4-4ea6-9dbb-4ee773b8905c-0', usage_metadata={'input_tokens': 15, 'output_tokens': 35, 'total_tokens': 50})]}}
----
